<a href="https://colab.research.google.com/github/Basel-byte/Network-Anomaly-Detection/blob/main/Pr_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practice Lab: Clustering  

In this exercise, we will know how K-Means and Normalized Cut algorithms can be used for network anomaly detection.

# Outline
- [ 1 - Extracting Dataset](#1)
- [ 2 - Packages ](#2)
- [ 3 - Reading data and Preprocessing](#3)
  - [3.1 Reading Columns' Names](#3.1)
  - [3.2 Reading dataset](#3.2)
  - [3.3 Removing class column](#3.3)
  - [3.4 Changing Categorical features to numericals](#3.4)
  
   
- [ 4 - K-means](#4)
- [ 5 - Spectral Clustering](#5)
  - [5.1 Getting Laplacian and Degree Matrices](#5.1)
  - [5.2 Spectral Clustering Algorithm](#5.2)
- [ 6 - New Clustering Algorithm](#6)
- [ 7 - Clustering Evaluation](#7)
  - [ 7.1 - Getting Predicted labels](#7.1)
  - [ 7.2 - Precision Measure](#7.2)
  - [ 7.3 - Recall Measure](#7.3)
  - [ 7.4 - F1 Measure](#7.4)
  - [ 7.5 - Entropy Measure](#7.5)
  

  

<a name="1"></a>
## 1 - Extracting Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# do not run if you have unzipped the dataset
!gzip -d /content/drive/MyDrive/kddcup.data.gz
!gzip -d /content/drive/MyDrive/corrected.gz

gzip: /content/drive/MyDrive/kddcup.data.gz: No such file or directory
gzip: /content/drive/MyDrive/corrected.gz: No such file or directory


<a name="2"></a>
## 2 - Packages 

First, let's run the cell below to import all the packages that you will need during this assignment.
- [numpy](https://numpy.org/) is the fundamental package for scientific computing with Python.
<!-- - [matplotlib](http://matplotlib.org) is a popular library to plot graphs in Python. -->
<!-- - [tensorflow](https://www.tensorflow.org/) a popular platform for machine learning. -->
- [pandas](https://pandas.pydata.org/) is open source data analysis and manipulation tool.

In [1]:
import numpy as np
import pandas as pd
import os
import requests
from enum import Enum
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import rbf_kernel
from scipy.linalg import eigh
from scipy.stats import mode
from sklearn.cluster import KMeans

<a name="3"></a>
## 3 - Reading data and Preprocessing

<a name="3.1"></a>
#### Reading columns' names 

In [2]:
DATASET_COLUMNS_FILE = "/content/drive/MyDrive/kddcup1999_columns.txt"
column_types =[]

with open(DATASET_COLUMNS_FILE, 'r') as file:
    column_labels: str = file.read()

column_regex: re.Pattern = re.compile(r"^(?P<column_name>\w+): (?P<data_type>\w+)\.$")
for column_type in column_labels.splitlines()[:]:
    match = column_regex.match(column_type)
    column_types.append(match.group("column_name"))
del column_labels

<a name="3.2"></a>
#### Reading the training dataset

In [3]:
data = pd.read_csv("/content/drive/MyDrive/kddcup.data", header=None)
data.columns = column_types

<a name="3.3"></a>
#### Removing class column

In [4]:
data_without_labels = data.drop(columns=["class"])
labels = data["class"]
del data
del column_types

<a name="3.4"></a>
### Change the categorical features to numericals

In [5]:
def convert_string_to_numeric(data_frame):
  # df_copy = data_frame.copy()
  for col in data_frame:
    if data_frame[col].dtypes == object:
      my_dict = {elem: index for index, elem in enumerate(data_frame[col].unique())}
      data_frame[col].replace(my_dict, inplace=True)
  return data_frame

In [6]:
def convert_string_to_binary_numeric(data_frame):
  # df_copy = data_frame.copy()
  for col in data_frame:
    if data_frame[col].dtypes == object:
      df1 = pd.get_dummies(data_frame[col])
      data_frame = pd.concat([data_frame, df1], axis=1).reindex(data_frame.index)
      data_frame.drop(col, axis=1, inplace=True)
  return data_frame

In [7]:
# use either of them but not both
# convert_string_to_numeric(data_without_labels.copy())

data_without_labels = convert_string_to_binary_numeric(data_without_labels).values.astype(np.float32)

<a name="4"></a>
## 4 - K-means



In [8]:
def Kmeans(data_frame_no_label, k = 3):
  return 

<a name="5"></a>
## 5 - Spectral clustering


<a name="5.1"> </a>
### 5.1 Getting Laplacian and Similarity Matrices

In [8]:
def get_laplacian_degree(data_no_label, gamma=0.1):
  sim_matrix = rbf_kernel(data_no_label, gamma=gamma)
  del data_no_label
  sim_matrix = np.add(sim_matrix, np.finfo(sim_matrix.dtype).eps)
  np.fill_diagonal(sim_matrix, 0)
  deg_matrix = np.sum(sim_matrix, axis=1)
  np.fill_diagonal(sim_matrix, deg_matrix)
  print(deg_matrix[deg_matrix == 0].shape, np.mean(deg_matrix))
  return sim_matrix, np.diag(deg_matrix)

<a name="5.2"></a>
### 5.2 Spectral Clustering Algorithm

In [9]:
def spectral_clustering(data_no_label, k = 3):
  data_no_label, dummy = train_test_split(data_no_label, train_size=0.0015, random_state=42, stratify=labels)
  del dummy
  laplacian_mat, degree_mat = get_laplacian_degree(data_no_label) # deg_mat is a vector
  eigenvalues, eigenvectors = eigh(laplacian_mat, degree_mat)
  del eigenvalues
  eigenvectors = eigenvectors[:, :k] / np.linalg.norm(eigenvectors[:, :k], ord=2, axis=1, keepdims=True)
  # to use the implemented kmeans, remove the next 3 lines and use the commented one
  kmeans = KMeans(n_clusters=k)
  kmeans.fit(eigenvectors)
  return kmeans.labels_
  # return Kmeans(eigenvectors, k)

<a name="6"></a> 
## 6-  New Clustering Algorithm

In [10]:
# def new_clustering_algorithm(data_frame_no_label, k = 3):
  

<a name="7"></a>
## 7- Clustering Evaluation

<a name="7.1"></a>
### 7.1 - getting predicted labels

In [10]:
c_labels = spectral_clustering(data_without_labels, 11)
unique_elements, counts = np.unique(c_labels, return_counts=True)
print(unique_elements, counts)

(0,) 1511.485


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[ 0  1  2  3  4  5  6  7  8  9 10] [ 713 1003 1268 1823  299  335  365  334  339  443  425]


In [11]:
# run this line in spectral clustering 
labels = train_test_split(labels, train_size=0.0015, random_state=42, stratify=labels)[0]
labels = convert_string_to_numeric(pd.DataFrame(labels)).values
labels = labels.reshape(labels.size)
counts = np.unique(labels, return_counts=True)[1]

<a name="7.2"></a>
### 7.2 - Recall Measure


In [12]:
def measure_recall(real_labels, predicted_labels):
  sum = 0;
  unique_predic = np.unique(predicted_labels)
  for c_label in unique_predic:
    cluster = real_labels[np.array(np.where(predicted_labels == c_label)[0].tolist())]
    major_class = mode(cluster, axis=None, keepdims=True)
    print(major_class.count[0] / counts[major_class.mode[0]])
    sum += major_class.count[0] / counts[major_class.mode[0]]
  return sum / unique_predic.size

print(measure_recall(labels, c_labels))

0.13892187128947994
0.22892424602232248
0.8259081562714188
0.4210401329850392
0.15609452736318408
0.13121890547263682
0.16480099502487564
0.06388031346473522
0.14676616915422885
0.1890547263681592
0.08097838993113275
0.23159894848611023


<a name="7.3"></a>
### 7.3 - Precision Measure

In [13]:
def measure_precision(real_labels, predicted_labels):
  sum = 0;
  unique_predic = np.unique(predicted_labels)
  for c_label in unique_predic:
    cluster = real_labels[np.array(np.where(predicted_labels == c_label)[0].tolist())]
    major_class = mode(cluster, axis=None, keepdims=True)
    print(major_class.count[0] / cluster.size)
    sum += major_class.count[0] / cluster.size
  return sum / unique_predic.size
print(measure_precision(labels, c_labels))

0.820476858345021
0.9611166500498505
0.9503154574132492
0.9725726823916621
0.8394648829431438
0.6298507462686567
0.726027397260274
0.8053892215568862
0.696165191740413
0.6862302483069977
0.8023529411764706
0.8081783888593297


<a name="7.4"></a>
### 7.4 F1 Score

In [14]:
def measure_f1(real_labels, predicted_labels):
  sum = 0;
  unique_predic = np.unique(predicted_labels)
  for c_label in unique_predic:
    cluster = real_labels[np.array(np.where(predicted_labels == c_label)[0].tolist())]
    major_class = mode(cluster, axis=None, keepdims=True)
    print(major_class.count[0] / cluster.size, ", ", major_class.count[0] / counts[major_class.mode[0]])
    recall = major_class.count[0] / counts[major_class.mode[0]]
    precis = major_class.count[0] / cluster.size
    sum += (2 * recall * precis) / (recall + precis)
  return sum / unique_predic.size
print(measure_f1(labels, c_labels))

0.820476858345021 ,  0.13892187128947994
0.9611166500498505 ,  0.22892424602232248
0.9503154574132492 ,  0.8259081562714188
0.9725726823916621 ,  0.4210401329850392
0.8394648829431438 ,  0.15609452736318408
0.6298507462686567 ,  0.13121890547263682
0.726027397260274 ,  0.16480099502487564
0.8053892215568862 ,  0.06388031346473522
0.696165191740413 ,  0.14676616915422885
0.6862302483069977 ,  0.1890547263681592
0.8023529411764706 ,  0.08097838993113275
0.33020125238533315


<a name="7.5"></a>
### 7.5 Conditional Entropy

In [15]:
def measure_entropy(real_labels, predicted_labels):
  unique_predic = np.unique(predicted_labels)
  total_entropy = 0
  for c_label in unique_predic:
    cluster = real_labels[np.array(np.where(predicted_labels == c_label)[0].tolist())]
    probs_in_cluster = np.unique(cluster, return_counts = True)[1] / cluster.size
    probs_in_cluster = probs_in_cluster * np.log2(probs_in_cluster) 
    cluster_cond_entropy = -np.sum(probs_in_cluster)
    print(cluster_cond_entropy)
    total_entropy += (cluster.size / real_labels.size * cluster_cond_entropy)
  return total_entropy
print(measure_entropy(labels, c_labels))

0.7339641112825588
0.2676780789172555
0.40371100466932563
0.19584011335798077
0.7602526629107298
1.455922181572364
1.3021367217721038
0.8455980506040853
1.2481424264547125
1.3178526112701772
0.9086228970510021
0.6161116721197659
